In [14]:
from collections import deque
import numpy as np
import pandas as pd
from pulp import *
from pulp import GLPK
import datetime as dt
from datetime import datetime, timedelta
import time
import math

Imprting Data from excel File using pandas lib

In [15]:
Data = pd.read_excel('Vol.xlsx', sheet_name='Vol')
Data.info

<bound method DataFrame.info of           DATE       HPA       HPD  Cie  Ligne  Esc 1  Immat Type app   PKG  \
0   2022-04-09       NaN  07:15:00  DAH   4000    TUN  7TVKD     B738   P18   
1   2022-04-09       NaN  07:30:00  DTH   1400    RNS  7TVCL     DH8D   PA1   
2   2022-04-09       NaN  07:30:00  DTH   1408    RNS  7TVCR     DH8D   PA1   
3   2022-04-09       NaN  08:00:00  DAH   6258    CZL  7TVUK    ATR72   PA1   
4   2022-04-09       NaN  08:00:00  DTH   2320    AZR  7TVCE     B738   PA1   
..         ...       ...       ...  ...    ...    ...    ...      ...   ...   
189 2022-04-10       NaN  18:40:00  DAH   6006  7TVKD   B738      AAE  P11A   
190 2022-04-10  19:55:00       NaN  DAH   1041  7TVKK   B738      NCE   P19   
191 2022-04-10  20:30:00       NaN  DAH   4003  7TVKM   B738      TUN   P15   
192 2022-04-10       NaN  22:45:00  DAH   4062  7TVJA   A330      DXB   P21   
193 2022-04-10  22:50:00       NaN  AZA    802    NaN   A320      FCO   P16   

          Remarque 

In [16]:
Data.head

<bound method NDFrame.head of           DATE       HPA       HPD  Cie  Ligne  Esc 1  Immat Type app   PKG  \
0   2022-04-09       NaN  07:15:00  DAH   4000    TUN  7TVKD     B738   P18   
1   2022-04-09       NaN  07:30:00  DTH   1400    RNS  7TVCL     DH8D   PA1   
2   2022-04-09       NaN  07:30:00  DTH   1408    RNS  7TVCR     DH8D   PA1   
3   2022-04-09       NaN  08:00:00  DAH   6258    CZL  7TVUK    ATR72   PA1   
4   2022-04-09       NaN  08:00:00  DTH   2320    AZR  7TVCE     B738   PA1   
..         ...       ...       ...  ...    ...    ...    ...      ...   ...   
189 2022-04-10       NaN  18:40:00  DAH   6006  7TVKD   B738      AAE  P11A   
190 2022-04-10  19:55:00       NaN  DAH   1041  7TVKK   B738      NCE   P19   
191 2022-04-10  20:30:00       NaN  DAH   4003  7TVKM   B738      TUN   P15   
192 2022-04-10       NaN  22:45:00  DAH   4062  7TVJA   A330      DXB   P21   
193 2022-04-10  22:50:00       NaN  AZA    802    NaN   A320      FCO   P16   

          Remarque   

In [17]:
Data.shape
Data= Data.astype({"DATE": str}, errors='raise')

We needed to make the data col as string because pands automaticly detaxted dates , but it made it deficult for us to process 

In [18]:
Data.info()
print(Data['DATE'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DATE      194 non-null    object
 1   HPA       95 non-null     object
 2   HPD       99 non-null     object
 3   Cie       194 non-null    object
 4   Ligne     194 non-null    int64 
 5   Esc 1     172 non-null    object
 6   Immat     180 non-null    object
 7   Type app  194 non-null    object
 8   PKG       194 non-null    object
 9   Remarque  194 non-null    object
 10  Comment   170 non-null    object
 11  Terminal  194 non-null    object
dtypes: int64(1), object(11)
memory usage: 18.3+ KB
0      2022-04-09
1      2022-04-09
2      2022-04-09
3      2022-04-09
4      2022-04-09
          ...    
189    2022-04-10
190    2022-04-10
191    2022-04-10
192    2022-04-10
193    2022-04-10
Name: DATE, Length: 194, dtype: object


In [19]:

Time=[]
for i in range (len(Data)):
    if pd.notna(Data['HPA'][i]) : #Convert heure prevu d'accostage and adding to her the exact date
        Time.append(str(Data["DATE"][i])+" "+str(Data['HPA'][i]))
    elif pd.notna(Data['HPD'][i]) :#Convert heure prevu d'essacostage and adding to her the exact date
        Time.append(str(Data["DATE"][i])+" "+str(Data['HPD'][i]))
    else:
        Time.append(float('nan'))

print(Time)
Data['DATETIME']=Time #creating a new col in the table with the datetime for the operation .
print(Data)

['2022-04-09 07:15:00', '2022-04-09 07:30:00', '2022-04-09 07:30:00', '2022-04-09 08:00:00', '2022-04-09 08:00:00', '2022-04-09 08:15:00', '2022-04-09 08:45:00', '2022-04-09 08:45:00', '2022-04-09 08:50:00', '2022-04-09 09:00:00', '2022-04-09 09:10:00', '2022-04-09 09:10:00', '2022-04-09 09:10:00', '2022-04-09 09:10:00', '2022-04-09 09:15:00', '2022-04-09 09:15:00', '2022-04-09 09:15:00', '2022-04-09 09:20:00', '2022-04-09 09:45:00', '2022-04-09 09:45:00', '2022-04-09 10:00:00', '2022-04-09 10:00:00', '2022-04-09 10:00:00', '2022-04-09 10:10:00', '2022-04-09 10:15:00', '2022-04-09 10:20:00', '2022-04-09 10:20:00', '2022-04-09 10:25:00', '2022-04-09 10:30:00', '2022-04-09 10:30:00', '2022-04-09 10:30:00', '2022-04-09 10:35:00', '2022-04-09 10:50:00', '2022-04-09 10:50:00', '2022-04-09 11:00:00', '2022-04-09 11:00:00', '2022-04-09 11:05:00', '2022-04-09 11:20:00', '2022-04-09 11:35:00', '2022-04-09 11:45:00', '2022-04-09 11:45:00', '2022-04-09 12:00:00', '2022-04-09 12:05:00', '2022-04-0

In [21]:

TimeGap=input('Give a Time GAP') #input gap time based on the need and calculation 
TimeGap=int(TimeGap)
def Trans_to_min(x):
    D=x.day*24*60
    min = x.minute
    H = x.hour *60
    return int(D+min+H)
# this fenction transfer the date time into a number , we transfered the date into munites also , because in the night shift , the 00:00 comes after 23:00 , so we needed another variable to tell us that the day changed

Graph_intervale=[]

for i in range(len(Data)):
    try:
        Graph_intervale.append([])
        Time[i] = dt.datetime.strptime(str(Time[i]), '%Y-%d-%m %H:%M:%S')
        min= Time[i]- dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[i].append(Trans_to_min(min))
        max= Time[i]+ dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[i].append(Trans_to_min(max))
    except:
        Time[i] = dt.datetime.strptime(str(Time[i]), '%d-%m-%Y %H:%M:%S')
        min= Time[i]- dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[i].append(Trans_to_min(min))
        max= Time[i]+ dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[i].append(Trans_to_min(max)) 
print((Graph_intervale))
Data['Graph_intervale']=Graph_intervale


[[6190, 6200], [6205, 6215], [6205, 6215], [6235, 6245], [6235, 6245], [6250, 6260], [6280, 6290], [6280, 6290], [6285, 6295], [6295, 6305], [6305, 6315], [6305, 6315], [6305, 6315], [6305, 6315], [6310, 6320], [6310, 6320], [6310, 6320], [6315, 6325], [6340, 6350], [6340, 6350], [6355, 6365], [6355, 6365], [6355, 6365], [6365, 6375], [6370, 6380], [6375, 6385], [6375, 6385], [6380, 6390], [6385, 6395], [6385, 6395], [6385, 6395], [6390, 6400], [6405, 6415], [6405, 6415], [6415, 6425], [6415, 6425], [6420, 6430], [6435, 6445], [6450, 6460], [6460, 6470], [6460, 6470], [6475, 6485], [6480, 6490], [6500, 6510], [6505, 6515], [6505, 6515], [6535, 6545], [6540, 6550], [6545, 6555], [6550, 6560], [6555, 6565], [6555, 6565], [6560, 6570], [6570, 6580], [6615, 6625], [6625, 6635], [6625, 6635], [6625, 6635], [6625, 6635], [6630, 6640], [6655, 6665], [6665, 6675], [6670, 6680], [6670, 6680], [6675, 6685], [6675, 6685], [6680, 6690], [6690, 6700], [6705, 6715], [6715, 6725], [6715, 6725], [6715

we transftered into munites every variables for us to create an interval of time .

In [22]:
Brigade=[]
for i in range (len(Data)):
    if (Time[i].hour >=0 and Time[i].hour<=7) or (Time[i].hour >=19 and Time[i].hour<=23): 
        Brigade.append("Nuit")
    else:
        Brigade.append("Journee")
Data['Brigade']=Brigade
print(Data)

           DATE       HPA       HPD  Cie  Ligne  Esc 1  Immat Type app   PKG  \
0    2022-04-09       NaN  07:15:00  DAH   4000    TUN  7TVKD     B738   P18   
1    2022-04-09       NaN  07:30:00  DTH   1400    RNS  7TVCL     DH8D   PA1   
2    2022-04-09       NaN  07:30:00  DTH   1408    RNS  7TVCR     DH8D   PA1   
3    2022-04-09       NaN  08:00:00  DAH   6258    CZL  7TVUK    ATR72   PA1   
4    2022-04-09       NaN  08:00:00  DTH   2320    AZR  7TVCE     B738   PA1   
..          ...       ...       ...  ...    ...    ...    ...      ...   ...   
189  2022-04-10       NaN  18:40:00  DAH   6006  7TVKD   B738      AAE  P11A   
190  2022-04-10  19:55:00       NaN  DAH   1041  7TVKK   B738      NCE   P19   
191  2022-04-10  20:30:00       NaN  DAH   4003  7TVKM   B738      TUN   P15   
192  2022-04-10       NaN  22:45:00  DAH   4062  7TVJA   A330      DXB   P21   
193  2022-04-10  22:50:00       NaN  AZA    802    NaN   A320      FCO   P16   

          Remarque                 Comm

In [23]:
#Select Brigate
Date=input('Input your the date you want')
Brigade=input('Input the brigade you want ')

In [24]:

if Brigade=='Journee':
    Selected_Data=Data.loc[Data['DATE'] == Date]
    Selected_Data=Selected_Data.loc[Selected_Data['Brigade'] == Brigade]
if Brigade=='Nuit':
    Selected_Data=Data.loc[Data['Brigade']==Brigade]
    Selected_Data=Selected_Data.loc[Selected_Data['Brigade'] == Brigade]

        

print(Selected_Data)


          DATE       HPA       HPD  Cie  Ligne Esc 1  Immat Type app   PKG  \
3   2022-04-09       NaN  08:00:00  DAH   6258   CZL  7TVUK    ATR72   PA1   
4   2022-04-09       NaN  08:00:00  DTH   2320   AZR  7TVCE     B738   PA1   
5   2022-04-09       NaN  08:15:00  DAH   6170   AAE  7TVUM    ATR72   PA1   
6   2022-04-09       NaN  08:45:00  DAH   6180   ORN  7TVUT    ATR72   PA1   
7   2022-04-09       NaN  08:45:00  DTH   1154   HME  7TVCD     B738  P10C   
..         ...       ...       ...  ...    ...   ...    ...      ...   ...   
91  2022-04-09  18:20:00       NaN  DAH   3017   IST  7TVKO     B738   P15   
92  2022-04-09       NaN  18:20:00  DAH   6026   CZL  7TVJU     B736  P11A   
93  2022-04-09       NaN  18:30:00  DAH   6106   ORN  7TVJS     B736    P9   
94  2022-04-09  18:40:00       NaN  DAH   6193   CZL  7TVKE     B738   P12   
95  2022-04-09       NaN  18:40:00  DAH   6006   AAE  7TVKL     B738  P10C   

         Remarque                  Comment Terminal            

Slection of the brigade based on the need , the pd.loc will filter based on the condition ,but the indexes will stay the same.

In [25]:
Graph_intervale=Selected_Data["Graph_intervale"]
Graph_intervale=np.array(Graph_intervale) #doing array to remove the indexes from the selections before , also sice we require a type list for our code to work 
print(Graph_intervale)

[list([6235, 6245]) list([6235, 6245]) list([6250, 6260])
 list([6280, 6290]) list([6280, 6290]) list([6285, 6295])
 list([6295, 6305]) list([6305, 6315]) list([6305, 6315])
 list([6305, 6315]) list([6305, 6315]) list([6310, 6320])
 list([6310, 6320]) list([6310, 6320]) list([6315, 6325])
 list([6340, 6350]) list([6340, 6350]) list([6355, 6365])
 list([6355, 6365]) list([6355, 6365]) list([6365, 6375])
 list([6370, 6380]) list([6375, 6385]) list([6375, 6385])
 list([6380, 6390]) list([6385, 6395]) list([6385, 6395])
 list([6385, 6395]) list([6390, 6400]) list([6405, 6415])
 list([6405, 6415]) list([6415, 6425]) list([6415, 6425])
 list([6420, 6430]) list([6435, 6445]) list([6450, 6460])
 list([6460, 6470]) list([6460, 6470]) list([6475, 6485])
 list([6480, 6490]) list([6500, 6510]) list([6505, 6515])
 list([6505, 6515]) list([6535, 6545]) list([6540, 6550])
 list([6545, 6555]) list([6550, 6560]) list([6555, 6565])
 list([6555, 6565]) list([6560, 6570]) list([6570, 6580])
 list([6615, 6

In [27]:
def Verify_in_interval(M1,M2):
    if M1[0]<=M2[0] and M1[1]>=M2[0]:
        return True
    elif M1[0]<=M2[1] and M1[1]>=M2[1]:
        return True
    else: 
        return False 
def   listAdj(M):
    listAdj=[]
    for i in range(len(M)):
        listAdj.append([])
        for j in [x for x in range(len(M)) if x != i]:
            if Verify_in_interval(M[i],M[j]) or Verify_in_interval(M[j],M[i]) :
                listAdj[i].append(j)
            else:
                pass
    return listAdj
            #print(listAdj[i])
    #print(listAdj[i])
# print(M)
Adj=listAdj(Graph_intervale)
print(Adj)

[[1], [0], [], [4, 5], [3, 5], [3, 4, 6], [5, 7, 8, 9, 10], [6, 8, 9, 10, 11, 12, 13, 14], [6, 7, 9, 10, 11, 12, 13, 14], [6, 7, 8, 10, 11, 12, 13, 14], [6, 7, 8, 9, 11, 12, 13, 14], [7, 8, 9, 10, 12, 13, 14], [7, 8, 9, 10, 11, 13, 14], [7, 8, 9, 10, 11, 12, 14], [7, 8, 9, 10, 11, 12, 13], [16], [15], [18, 19, 20], [17, 19, 20], [17, 18, 20], [17, 18, 19, 21, 22, 23], [20, 22, 23, 24], [20, 21, 23, 24, 25, 26, 27], [20, 21, 22, 24, 25, 26, 27], [21, 22, 23, 25, 26, 27, 28], [22, 23, 24, 26, 27, 28], [22, 23, 24, 25, 27, 28], [22, 23, 24, 25, 26, 28], [24, 25, 26, 27], [30, 31, 32], [29, 31, 32], [29, 30, 32, 33], [29, 30, 31, 33], [31, 32], [], [36, 37], [35, 37], [35, 36], [39], [38], [41, 42], [40, 42], [40, 41], [44, 45], [43, 45, 46], [43, 44, 46, 47, 48], [44, 45, 47, 48, 49], [45, 46, 48, 49], [45, 46, 47, 49], [46, 47, 48, 50], [49], [52, 53, 54, 55], [51, 53, 54, 55, 56], [51, 52, 54, 55, 56], [51, 52, 53, 55, 56], [51, 52, 53, 54, 56], [52, 53, 54, 55], [58], [57, 59, 60, 61, 

In [28]:
def matriceAdj(Adj):
    Mat=[]
    for i in range( len(Adj)):
        Mat.append([])
        for j in range( len(Adj)):
            Mat[i].append(0)
            for k in range(len(Adj[i])):
                if j==Adj[i][k]:
                    Mat[i][j]=1
    return Mat

In [30]:
def arret(M):
    ex=[[],[]]
    for i in range (len(M)-1):
        for j in range(i+1,len(M)):
            if M[i][j]==1:
                ex[0].append(i)
                ex[1].append(j)
    return ex
#Create a list of start and end of each arret in our graph

In [31]:
Mat=matriceAdj(Adj)
Mat= np.array(Mat)
print (Mat)

[[0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 1 1 0]]


In [32]:
Ex=arret(Mat)
print("ex1= ",Ex[0])
print("ex2= ",Ex[1])

ex1=  [0, 3, 3, 4, 5, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 12, 12, 13, 15, 17, 17, 17, 18, 18, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25, 25, 25, 26, 26, 27, 29, 29, 29, 30, 30, 31, 31, 32, 35, 35, 36, 38, 40, 40, 41, 43, 43, 44, 44, 45, 45, 45, 46, 46, 46, 47, 47, 48, 49, 51, 51, 51, 51, 52, 52, 52, 52, 53, 53, 53, 54, 54, 55, 57, 58, 58, 58, 58, 59, 59, 59, 59, 60, 60, 60, 61, 61, 62, 63, 65, 65, 65, 66, 66, 66, 67, 67, 68, 69, 71, 71, 71, 72, 72, 73, 73, 73, 74, 74, 75, 75, 76, 77, 77, 78, 78, 79, 80, 81, 81, 82, 82, 83, 85, 85, 86, 88, 88, 89, 90, 90, 91]
ex2=  [1, 4, 5, 5, 6, 7, 8, 9, 10, 8, 9, 10, 11, 12, 13, 14, 9, 10, 11, 12, 13, 14, 10, 11, 12, 13, 14, 11, 12, 13, 14, 12, 13, 14, 13, 14, 14, 16, 18, 19, 20, 19, 20, 20, 21, 22, 23, 22, 23, 24, 23, 24, 25, 26, 27, 24, 25, 26, 27, 25, 26, 27, 28, 26, 27, 28, 27, 28, 28, 30, 31, 32, 31, 32, 32, 33, 33, 36, 37, 37, 39, 41, 42, 42, 44, 

In [33]:
def Gloton(Adj):
    n=len(Adj)
    NC=dict()
    C=[0 for k in range(n)]
    for i in range(n):
        NC[i]=[]
    def  rajouter(k,NC):
        if ((k in NC)==True):
            return NC
        else:
            NC.append(k)
            return NC
    i=0
    while i<=n-1:
        exist=True
        k=0
        while (exist==True):
            exist=k in NC[i]
            if (exist==True):
                k+=1
        C[i]=k    
        for j in range(len(Adj[i])):
            voisin=Adj[i][j]
            NC[voisin]=rajouter(k,NC[voisin])
        i+=1
    return C
C=Gloton(Adj)
print(C)
max_Conducteur=0
for i in C:
    if max_Conducteur<=i:
        max_Conducteur=i
max_Conducteur+=1
print(max_Conducteur)
max_Conducteur=input('Give the number of drivers')
max_Conducteur=int(max_Conducteur)

[0, 1, 0, 0, 1, 2, 0, 1, 2, 3, 4, 0, 5, 6, 7, 0, 1, 0, 1, 2, 3, 0, 1, 2, 3, 0, 4, 5, 1, 0, 1, 2, 3, 0, 0, 0, 1, 2, 0, 1, 0, 1, 2, 0, 1, 2, 0, 1, 3, 2, 0, 0, 1, 2, 3, 4, 0, 0, 1, 0, 2, 3, 4, 1, 0, 0, 1, 2, 3, 0, 1, 0, 1, 2, 3, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 0, 1, 2, 0, 1, 2, 0, 1]
8


In [35]:
# %%
# Import of the pyomo module
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ

# Creation of a Concrete Model
model = ConcreteModel()

# %%
model.C = Set(initialize=range(max_Conducteur), doc='les Conducteur')
model.J = Set(initialize=range(len(Adj)), doc='les jobs')
model.A = Set(initialize=range(len(Ex[1])), doc='les arcs')

# %%
model.n = Param(initialize=len(Adj), doc='le nombre de jobs')
model.c = Param(initialize=max_Conducteur, doc='le nombre de conducteur')
model.m = Param(initialize=len(Ex[1]), doc='les relations entre les arcs')
model.ex1 = Param(model.A,initialize=Ex[0], doc='extrémité 1')
model.ex2 = Param(model.A,initialize=Ex[1], doc='extrémité 2')

# %%
model.x = Var(model.C, model.J,domain=Binary, doc='affectation conducteur')
model.v = Var(model.C,bounds=(0,None),domain=Integers, doc='les taches')
model.y = Var(bounds=(0,None),domain=Integers, doc='valeur de comparaison')

# %%
def Cotraint1(model,j):
  return sum(model.x[i,j] for i in model.C) == 1
model.C1 = Constraint(model.J, rule=Cotraint1, doc='verification daffectation')


def Contraint2(model,i,k):
  return ((model.x[i,model.ex1[k]]+model.x[i,model.ex2[k]])) <= 1  

model.C2 = Constraint (model.C,model.J, rule=Contraint2, doc='verification de non chevauchement ')

def Contraint3(model,i):
  return sum(model.x[i,j] for j in model.J)==model.v[i]

model.C3 = Constraint (model.C, rule=Contraint3, doc='une seul tache par conducteur')

def Contraint4(model,i):
  return (model.v[i]<=model.y)

model.C4 = Constraint (model.C, rule=Contraint4, doc='maximiser')

# %%
def objective_rule(model):
  return model.y
model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')

# %%
def pyomo_postprocess(options=None, instance=None, results=None):
  model.x.display()
# %%


opt = SolverFactory("gurobi")
results = opt.solve(model,tee=True)
#sends results to stdout
results.write()
print("\nDisplaying Solution\n" + '-'*60)
pyomo_postprocess(None, model, results)
# %%




Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-10
Read LP format model from file C:\Users\pc\AppData\Local\Temp\tmpt0ikrqkg.pyomo.lp
Reading time = 0.01 seconds
x754: 854 rows, 754 columns, 3001 nonzeros
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 854 rows, 754 columns and 3001 nonzeros
Model fingerprint: 0x19a2db45
Variable types: 1 continuous, 753 integer (744 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 16.0000000
Presolve removed 593 rows and 9 columns
Presolve time: 0.02s
Presolved: 261 rows, 745 columns, 2072 nonzeros
Variable types: 0 continuous, 745 integer (744 binary)

Root relaxation: objective 1.162500e+01, 173 iterations, 0.00 seconds (0.00 work units)

    Nodes    |

In [36]:
model.v.display()
#verfy how much each driver is working per day

v : les taches
    Size=8, Index=C
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  11.0 :  None : False : False : Integers
      1 :     0 :  12.0 :  None : False : False : Integers
      2 :     0 :  12.0 :  None : False : False : Integers
      3 :     0 :  12.0 :  None : False : False : Integers
      4 :     0 :  12.0 :  None : False : False : Integers
      5 :     0 :  12.0 :  None : False : False : Integers
      6 :     0 :  10.0 :  None : False : False : Integers
      7 :     0 :  12.0 :  None : False : False : Integers


In [51]:
x=model.x
Group_co=[]
for i in model.C:
    Group_co.append([])
    for j in model.J:
        if x[i,j].value == 1:
            Group_co[i].append(j)

print(Group_co)

Slected_DATETIME= np.array(Selected_Data['DATETIME'])
Slected_Terminal=np.array(Selected_Data['Terminal'])
Slected_PKG=np.array(Selected_Data['PKG'])
#Slected_HPA=np.array(Selected_Data['HPA'])
#Slected_HPD=np.array(Selected_Data['HPD'])
for i in range(len(Group_co)):
    print("list des vols du conducteur "+str(i+1))
    print("Terminal       N Parking          Time")

    for j in Group_co[i]:
        print(Slected_Terminal[j]+'  |  '+Slected_PKG[j]+'  |  '+Slected_DATETIME[j])
        




[[0, 9, 28, 36, 40, 53, 58, 60, 64, 80, 85], [10, 15, 18, 22, 34, 41, 59, 65, 67, 78, 86, 91], [1, 3, 14, 16, 19, 25, 33, 57, 61, 62, 69, 81], [13, 27, 32, 35, 46, 49, 56, 71, 74, 76, 84, 92], [4, 8, 20, 42, 44, 47, 50, 55, 70, 72, 87, 89], [6, 12, 17, 24, 31, 43, 52, 63, 79, 82, 83, 88], [7, 21, 26, 30, 37, 38, 48, 54, 77, 90], [2, 5, 11, 23, 29, 39, 45, 51, 66, 68, 73, 75]]
list des vols du conducteur 1
Terminal       N Parking          Time
T2  |  PA1  |  2022-04-09 08:00:00
T4  |  P20  |  2022-04-09 09:10:00
T2  |  PA1  |  2022-04-09 10:35:00
T1  |  P3B  |  2022-04-09 11:45:00
T4  |  P16  |  2022-04-09 12:25:00
T4  |  P23B  |  2022-04-09 14:30:00
T4  |  P20  |  2022-04-09 15:10:00
T2  |  PA1  |  2022-04-09 15:15:00
T2  |  PA1  |  2022-04-09 15:35:00
T2  |  P9  |  2022-04-09 17:20:00
T4  |  P21  |  2022-04-09 18:00:00
list des vols du conducteur 2
Terminal       N Parking          Time
T2  |  PA1  |  2022-04-09 09:10:00
T2  |  PA1  |  2022-04-09 09:45:00
T2  |  PA1  |  2022-04-09 10